#  LSTM
## 로이터 뉴스 분류하기 실습

In [1]:
from keras.datasets import reuters

In [14]:
help(reuters)

Help on module keras.datasets.reuters in keras.datasets:

NAME
    keras.datasets.reuters - Reuters topic classification dataset.

FUNCTIONS
    get_word_index(path='reuters_word_index.json')
        Retrieves a dict mapping words to their index in the Reuters dataset.
        
        Args:
            path: where to cache the data (relative to `~/.keras/dataset`).
        
        Returns:
            The word index dictionary. Keys are word strings, values are their index.
    
    load_data(path='reuters.npz', num_words=None, skip_top=0, maxlen=None, test_split=0.2, seed=113, start_char=1, oov_char=2, index_from=3, **kwargs)
        Loads the Reuters newswire classification dataset.
        
        This is a dataset of 11,228 newswires from Reuters, labeled over 46 topics.
        
        This was originally generated by parsing and preprocessing the classic
        Reuters-21578 dataset, but the preprocessing code is no longer packaged
        with Keras. See this
        [githu

In [ ]:
#데이터 로드

In [3]:
(X_train, y_train),(X_test, y_test) = reuters.load_data(num_words=10000, test_split=0.3)

C:\ProgramData\Anaconda3\lib\site-packages\keras\datasets\reuters.py:143: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  x_train, y_train = np.array(xs[:idx]), np.array(labels[:idx])
C:\ProgramData\Anaconda3\lib\site-packages\keras\datasets\reuters.py:144: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  x_test, y_test = np.array(xs[idx:]), np.array(labels[idx:])


In [ ]:
#데이터 확인

In [17]:
X_train.shape

(7859,)

In [18]:
X_test.shape

(3369,)

In [19]:
import numpy as np
np.unique(y_train)
#0과1 만으로 이루어진 binary class가 아님 -> 다중클래스 

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
       34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45], dtype=int64)

In [20]:
data1 = reuters.get_word_index()
data1

{'mdbl': 10996,
 'fawc': 16260,
 'degussa': 12089,
 'woods': 8803,
 'hanging': 13796,
 'localized': 20672,
 'sation': 20673,
 'chanthaburi': 20675,
 'refunding': 10997,
 'hermann': 8804,
 'passsengers': 20676,
 'stipulate': 20677,
 'heublein': 8352,
 'screaming': 20713,
 'tcby': 16261,
 'four': 185,
 'grains': 1642,
 'broiler': 20680,
 'wooden': 12090,
 'wednesday': 1220,
 'highveld': 13797,
 'duffour': 7593,
 '0053': 20681,
 'elections': 3914,
 '270': 2563,
 '271': 3551,
 '272': 5113,
 '273': 3552,
 '274': 3400,
 'rudman': 7975,
 '276': 3401,
 '277': 3478,
 '278': 3632,
 '279': 4309,
 'dormancy': 9381,
 'errors': 7247,
 'deferred': 3086,
 'sptnd': 20683,
 'cooking': 8805,
 'stratabit': 20684,
 'designing': 16262,
 'metalurgicos': 20685,
 'databank': 13798,
 '300er': 20686,
 'shocks': 20687,
 'nawg': 7972,
 'tnta': 20688,
 'perforations': 20689,
 'affiliates': 2891,
 '27p': 20690,
 'ching': 16263,
 'china': 595,
 'wagyu': 16264,
 'affiliated': 3189,
 'chino': 16265,
 'chinh': 16266,
 '

In [21]:
data2 = {}
for k , v in data1.items():
    data2[v+3] = k
data2

{10999: 'mdbl',
 16263: 'fawc',
 12092: 'degussa',
 8806: 'woods',
 13799: 'hanging',
 20675: 'localized',
 20676: 'sation',
 20678: 'chanthaburi',
 11000: 'refunding',
 8807: 'hermann',
 20679: 'passsengers',
 20680: 'stipulate',
 8355: 'heublein',
 20716: 'screaming',
 16264: 'tcby',
 188: 'four',
 1645: 'grains',
 20683: 'broiler',
 12093: 'wooden',
 1223: 'wednesday',
 13800: 'highveld',
 7596: 'duffour',
 20684: '0053',
 3917: 'elections',
 2566: '270',
 3554: '271',
 5116: '272',
 3555: '273',
 3403: '274',
 7978: 'rudman',
 3404: '276',
 3481: '277',
 3635: '278',
 4312: '279',
 9384: 'dormancy',
 7250: 'errors',
 3089: 'deferred',
 20686: 'sptnd',
 8808: 'cooking',
 20687: 'stratabit',
 16265: 'designing',
 20688: 'metalurgicos',
 13801: 'databank',
 20689: '300er',
 20690: 'shocks',
 7975: 'nawg',
 20691: 'tnta',
 20692: 'perforations',
 2894: 'affiliates',
 20693: '27p',
 16266: 'ching',
 598: 'china',
 16267: 'wagyu',
 3192: 'affiliated',
 16268: 'chino',
 16269: 'chinh',
 2

In [22]:
#토크마이닝-텍스트 처리 
for idx, word in enumerate(("<pad>","<sos>","<unk>")):
    data2[idx] = word
    
#padding(빈공간) --> 0, 
#sos(문서의 시작부분) --> 1, 
#unknown(모르겠는 단어) --> 2 로 바꿔주겠다
#데이터가 4부터 있으므로 0,1,2를 주어도 괜찮겠다.

In [23]:
print(" ".join([data2[i] for i in X_train[1000]]))

<sos> german banking authorities are weighing rules for banks' off balance sheet activities in an attempt to cope with the growing volume of sophisticated capital market instruments banking sources said interest rate and currency swaps and currency options are under <unk> scrutiny and if revisions are made they may <unk> regulation jointly proposed by the u s and u k to japan juergen <unk> director of the bundesbank's division of banking law and credit <unk> said the u s british proposals were interesting but declined to elaborate but banking sources said west germany was more likely to produce its own conclusions than to adopt foreign proposals there is no formal plan yet but talks are in the latter stages one representative of the german banking association in <unk> said bankers expect rule changes this year all <unk> must be approved by the bundesbank west germany's four major banking associations and the federal banking supervisory office talks have been slowed by the fact that fun

In [ ]:
#데이터 준비

In [4]:
from keras.preprocessing import sequence
X_train = sequence.pad_sequences(X_train, maxlen=100)
X_test = sequence.pad_sequences(X_test, maxlen=100)

In [25]:
#다중클래스이므로 to_catrgorical
from tensorflow.keras.utils import to_categorical
y_train = to_categorical(y_train, 46)
y_test = to_categorical(y_test, 46)

In [28]:
#인공신경망 모형 구성
from keras.layers.embeddings import Embedding
from keras.models import Sequential
from keras.layers import LSTM, Dense

model = Sequential()
model.add(Embedding(input_dim = 10000,
                   output_dim =32,
                   input_length=100)) #X_train.shape
model.add(LSTM(units=100))
model.add(Dense(units=46, activation = 'softmax'))
#binary class - 원핫인코딩 필요 없음
# 이번 데이터는 다중클래스 - 분류 - softmax
#데이터의 특성을 잘 파악하고 있어야함 

In [29]:
#설정값 주입 
model.compile(loss='categorical_crossentropy',
             optimizer='rmsprop',
             metrics=['accuracy'])
#다중클래스
#adam or rmsprop --> 둘중 뭐가 좋을지 선택

In [ ]:
#학습수행

In [30]:
model.fit(X_train, y_train, epochs=10, batch_size=64, verbose=2)

Epoch 1/10
123/123 - 63s - loss: 2.4119 - accuracy: 0.3724
Epoch 2/10
123/123 - 45s - loss: 1.9973 - accuracy: 0.4774
Epoch 3/10
123/123 - 53s - loss: 1.7995 - accuracy: 0.5232
Epoch 4/10
123/123 - 47s - loss: 1.6830 - accuracy: 0.5464
Epoch 5/10
123/123 - 53s - loss: 1.5854 - accuracy: 0.5730
Epoch 6/10
123/123 - 52s - loss: 1.4603 - accuracy: 0.6086
Epoch 7/10
123/123 - 62s - loss: 1.3548 - accuracy: 0.6351
Epoch 8/10
123/123 - 66s - loss: 1.2718 - accuracy: 0.6519
Epoch 9/10
123/123 - 49s - loss: 1.2112 - accuracy: 0.6694
Epoch 10/10
123/123 - 46s - loss: 1.1161 - accuracy: 0.6927


In [31]:
loss, acc = model. evaluate(X_test, y_test)
print("손실값:",round(loss,3))
print("정확도:",round(acc,3))

106/106 [==============================] - 11s 65ms/step - loss: 1.4503 - accuracy: 0.6260
손실값: 1.45
정확도: 0.626


In [32]:
y_pred=model.predict(X_test)
print(y_pred[100:103])

[[1.90898572e-05 9.55273281e-05 7.02407897e-06 1.18018230e-02
  9.82521653e-01 7.78032700e-07 2.12172527e-05 9.19693684e-06
  1.03742997e-04 1.50519454e-05 2.08604997e-05 6.04947491e-05
  4.57563146e-06 3.78511439e-04 2.84461225e-06 3.30781063e-06
  3.18766269e-03 4.81155381e-04 1.35244704e-06 3.94789968e-04
  2.18113782e-04 1.67152393e-05 3.52969778e-07 1.15839568e-04
  7.32991339e-06 1.04990017e-04 3.15190118e-06 1.53768306e-05
  1.19288679e-05 5.12505540e-06 9.10413110e-06 6.46433764e-05
  1.57381655e-05 4.69040197e-06 4.69041970e-06 2.01384910e-06
  6.11420637e-06 6.87739021e-06 1.11511890e-05 1.17368800e-04
  1.21799785e-05 1.11093308e-04 1.87364390e-06 4.33371753e-07
  5.48114940e-07 1.93403093e-06]
 [4.61307463e-05 2.22373958e-04 1.81539344e-05 1.28048006e-02
  9.77586925e-01 2.41452312e-06 5.73677316e-05 2.08038655e-05
  1.67669627e-04 3.40546358e-05 4.56721646e-05 1.20510544e-04
  9.85223596e-06 7.42798729e-04 7.72162730e-06 8.35023184e-06
  4.61184792e-03 8.44611670e-04 3.409

In [33]:
y_pred_class = model.predict_classes(X_test)
print(y_pred_class[100:103])

C:\ProgramData\Anaconda3\lib\site-packages\keras\engine\sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


[4 4 3]
